# Utilizzo del pacchetto mltools per il preprocessing e l'analisi dei testi

In [ ]:
import pandas as pd
import numpy as np
import re
from bokeh.io import output_notebook
output_notebook()
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%reload_ext autoreload
%autoreload 2

Importiamo il dataset contenente i testi che intediamo analizzare:

In [ ]:
df = pd.read_csv("data/20newsgroup_body.csv")

In [ ]:
df.head()

Verifichiamo la presenza di dati mancanti, in questo caso documenti vuoti senza testo: notiamo che ne sono presenti alcuni nel nostro dataset, per questo motivo provvediamo ad eliminarli.

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

Vediamo ora come utilizzare il pacchetto mltools per l'analisi dei testi. Tramite l'import del modulo textMining è possibile utilizzare alcune classi per il preprocessing dei testi, al fine di rendere questi in forma eligibile per i modelli di classificazione. 

In [ ]:
print('{} classi trovate\nClassi:\n{}'.format(len(df['class'].unique()), df['class'].unique()))

In [ ]:
from mltools.textMining import TextPreprocessing

Istanziamo la classe text_preprocessing: in questo caso impostiamo il flag per effettuare la lemmatization uguale a False (i tempi computazionali diventano lunghi se True); il metodo che andremo poi a fittare sui dati andrà in questo caso ad effettuare una rimozione dei caratteri speciali (conserva solo le parole) e delle stopwords presenti nel testo. L'output sarà un pandas DataFrame uguale all'originale, ma con l'aggiunta di una colonna contenente i token estratti.

In [ ]:
tp = TextPreprocessing(lemmatization = False)

A questo punto possiamo estrarre i token prensenti nei documenti, passando al metodo *fit* il dataframe contenente l'estratto dei testi e il nome del campo che intendiamo processare:

In [ ]:
output = tp.fit(df, "text")

In [ ]:
output.head()

### Vectorization del testo

Passiamo ora a trasformare i token in una matrice di feature che potrà essere poi utilizzata dai modelli predittivi per la classificazione dei testi. In questo caso andiamo ad utilizzare come metrica per rappresentare le parole, il tf-idf.

In [ ]:
#suddivisione training set / test set
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(output[["tokens", "filename"]], output["class"], 
                                                    test_size=0.3, 
                                                    random_state=40)

Passiamo ora al metodo per vettorizzare le parole le pandas Series contenti i token estratti in precedenza:

In [ ]:
from mltools.textMining.textProcessing import VectorizeData

In [ ]:
token_toVect = VectorizeData()

In [ ]:
X_train_tfidf, X_test_tfidf, vectorizer = token_toVect.fit(X_train["tokens"], X_test["tokens"])

In [ ]:
X_train_tfidf.shape

In [ ]:
X_test_tfidf.shape

In [ ]:
vectorizer

### Estrazione dei topic

In [ ]:
no_features = 1000
token_toVect = VectorizeData(method='count')
X_train_count, X_test_count, count_vectorizer = token_toVect.fit(X_train["tokens"],
                                                        X_test["tokens"])
count_feature_names = count_vectorizer.get_feature_names()

#### Gensim

In [ ]:
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel

In [ ]:
# Create a corpus from a list of texts
texts = list(X_train['tokens'])
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# Train the model on the corpus.
lda = LdaModel(corpus, num_topics=20, id2word = dictionary)

In [ ]:
lda.show_topics()

In [ ]:
# Visualize the topics 
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary) 

In [ ]:
import pyLDAvis
pyLDAvis.show(vis)

#### SKLearn

In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
no_topics = 20

In [ ]:
# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
lda = LDA(n_components=no_topics, max_iter=5,
          learning_method='online', random_state=123)
lda_output = lda.fit_transform(X_train_count)

In [ ]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic_model in enumerate(model.components_):
        print ("\nTopic #%d:" % (topic_idx))
        print (" ".join([feature_names[i]
                         for i in topic_model.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
no_top_words = 10
display_topics(lda, count_feature_names, no_top_words)

Un modello con una alta log-likelihood e bassa perplexity [exp(-1. * log-likelihood per word)] è considerato un buon modello. Controlliamo il nostro modello.

In [ ]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda.score(X_train_count))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda.perplexity(X_train_count))

Effettuiamo una GridSearch sui parametri

In [ ]:
# Creiamo una nuova istanza con il nuovo modello
lda = LDA(learning_decay= 0.7, learning_method='online',
          n_components= 10,random_state=0)

In [ ]:
# get topics for some given samples:
lda_output = lda.fit_transform(X_train_count)
lda_output = lda.fit_transform(X_test_count)

In [ ]:
def make_topic_dataframe(model, matrix, limit_row):
    # column names
    topicnames = ["Topic" + str(i) for i in range(model.n_components)]

    # index names
    docnames = ["Doc" + str(i) for i in range(matrix.shape[0])]

    # Make the pandas dataframe
    df_document_topic = pd.DataFrame(np.round(matrix, 2), columns=topicnames, index=docnames)

    # Get dominant topic for each document
    dominant_topic = np.argmax(df_document_topic.values, axis=1)
    df_document_topic['dominant_topic'] = dominant_topic
    df_document_topics = df_document_topic.head(limit_row).style.applymap(color_green).applymap(make_bold)
    return df_document_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

In [ ]:
make_topic_dataframe(lda, X_train_count)

In [ ]:
import pyLDAvis.sklearn
vis = pyLDAvis.sklearn.prepare(lda, X_train_count, count_vectorizer)

In [ ]:
import pyLDAvis
pyLDAvis.show(vis)

### Word Embeddings

In [ ]:
from mltools.textMining.word2vec import *

In [ ]:
word2vec_model = load_model("GoogleNews-vectors-negative300.bin.gz")

In [ ]:
embeddings = get_word2vec_embeddings(word2vec_model, output, "tokens")

In [ ]:
np.array(embeddings)

### Classificazione

A questo punto, utilizziamo il modello evaluateModels per effettuare una CrossValidation e identificare i modelli migliori: 

In [ ]:
from mltools.evaluateModels import CrossValidation

In [ ]:
cv = CrossValidation(models=["MultinomialNB", "LogisticRegression"], 
                     scores = ["accuracy", "f1_multiclass", "precision_multiclass", "recall_multiclass"],
                     params_file = "./param_file.txt")

In [ ]:
cv.get_models_info()

In [ ]:
cv.get_scores_info()

In [ ]:
res, model = cv.fit_cv(X_train_tfidf, y_train)

In [ ]:
res

In [ ]:
from sklearn.metrics import accuracy_score, auc, f1_score, precision_score, recall_score, matthews_corrcoef

nb_clf = model['MultinomialNB']
nb_clf = nb_clf.fit(X_train_tfidf, y_train) 
y_predicted = nb_clf.predict(X_test_tfidf)

print("Accuratezza di MultinomialNB su test set:", accuracy_score(y_test, y_predicted))

### Analisi dell'importanza delle parole nella classificazione

In [ ]:
from mltools.textMining import *

In [ ]:
lgr_clf = model['LogisticRegression']
lgr_clf = lgr_clf.fit(X_train_tfidf, y_train)

In [ ]:
importance = get_most_important_features(vectorizer, lgr_clf, n=10)

In [ ]:
plot_important_words(importance, lgr_clf.classes_)

### Analisi della frequenza della parole nei documenti

In [ ]:
from mltools.textMining.featuresImportance import plot_word_freq

In [ ]:
plot_word_freq(output, target="class", col="tokens")

## BBC NEWS SUMMARIZATION 

### Creazione del dataset

Come prima cosa, importiamo le librerie necessarie e creiamo il dataset per l'analisi, leggendo i file contenenti gli articoli della BBC relativi all'area di business e i riassunti corrispondenti. 

In [ ]:
import pandas as pd
import numpy as np
import re
import os
import sys
import codecs
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path_news = "./data/BBC_Business_News/business_news"
path_summary = "./data/BBC_Business_News/business_summary"

In [ ]:
def create_df(path, column_name):
    dataset = pd.DataFrame()
    filename_list = os.listdir(path)
    for filename in filename_list:
        with codecs.open("{}/{}".format(path, filename), "r", encoding='utf-8', errors='ignore') as file:
            text = file.readlines()
            clean_text = [re.sub(r'(\n)', '', line) for line in text]
            complete_text = " ".join(clean_text)
            df = np.array(complete_text).reshape(1,1)
        dataset = pd.concat([dataset, pd.DataFrame(df)])
        
    dataset.columns = [column_name]
    dataset.index = np.arange(len(dataset))
    return dataset

In [ ]:
dataset_news = create_df(path_news, 'article')
dataset_summaries = create_df(path_summary, 'reference_summary')

In [ ]:
dataset_finale = pd.concat([dataset_news, dataset_summaries], axis = 1)
dataset_finale.head()

In [ ]:
dataset_finale = dataset_finale[dataset_finale['article'].apply(len) > 100]

In [ ]:
dataset_news['article'][0]

In [ ]:
dataset_finale.shape

### Summarization

L'obiettivo della 'Summarization' è creare un riassunto quanto più rappresentativo possibile dell'intero documento.

In particolare, la classe implementata utilizza i cosiddetti metodi 'extraction-based', che lavorano selezionando un sottoinsieme delle frasi più importanti esistenti nel testo originario per comporre il riassunto. 

Gli algoritmi che possono essere testati, restituiti dal metodo 'getInfo_models', sono i seguenti:
- TextRank (gensim e sumy);
- LexRank (sumy);
- Lsa (sumy);
- Luhn (sumy). 

Inoltre, settando il parametro 'keywords' uguale a True, la classe estrae le keywords dal testo in input.

Come prima cosa prima di procedere con la generazione dei riassunti, è necessario filtrare il campo del testo per eliminare gli eventuali campi che contengono stringhe vuote oppure testi con un numero troppo piccolo di frasi:

In [ ]:
from mltools.textMining import Summarization

In [ ]:
Summarization.getInfo_models()

In [ ]:
dataset_finale = dataset_finale[dataset_finale['article'].apply(Summarization.count_sentences) > 5]

In [ ]:
dataset_finale.shape

In [ ]:
SUM = Summarization(models = ['textrank-g', 'lexrank-s', 'lsa-s'])

In [ ]:
df_summary = SUM.fit(dataset_finale, field = 'article', n_sentences = 3)
df_summary.head()

In [ ]:
df_summary.loc[0]['article']

In [ ]:
df_summary.loc[0]['textrank-g_summary']

In [ ]:
df_summary.loc[0]['lexrank-s_summary']

### Evaluation

Il modo più comune per valutare il contenuto informativo di un riassunto automatico è di confrontarlo con un riassunto scritto dall'uomo.

Una delle metriche maggiormente usate a tale scopo è il ROUGE, che essenzialmente calcola le sovrapposizioni degli n-grammi tra i due riassunti messi a confronto. Un elevato grado di sovrapposizione dovrebbe indicare un alto livello di concetti condivisi tra i due riassunti.

Questo tipo di metrica, tuttavia, non riesce a fornire nessun feedback sulla coerenza dell'abstract. 

Le metriche che abbiamo scelto sono: 
- Rouge-1 (one-grams);
- Rouge-2 (bi-grams);
- Rouge-L (Longest Common Subsequence).

Per ognuna di esse, settando il parametro 'type_metric' uguale ad 'f', 'p' o 'r', si ha rispettivamente l'f-score, la precisione o la recall (di default è 'f').

In [ ]:
dictionary, df_scores = SUM.evaluate(df_summary, field_summary = 'reference_summary',
                            metrics = ['rouge-1', 'rouge-2', 'rouge-l'], type_metric = 'r')

Gli output di 'evaluate' sono:
- un dizionario di dataframes, uno per ogni metrica scelta. Il singolo dataframe contiene gli scores calcolati per ogni modello;
- un dataframe riassuntivo con le medie e le deviazioni standard degli scores per ogni metrica e ogni modello usati.

In [ ]:
dictionary.keys()

In [ ]:
dictionary['rouge-1_r_df'].head()

In [ ]:
print("Dataframe con la media e la deviazione standard degli scores per ogni metrica scelta:\n")
df_scores

Vediamo ora come estrarre le parole chiave da un testo:

In [ ]:
from mltools.textMining import Keywords

In [ ]:
kw = Keywords()

In [ ]:
kw_res = kw.fit(dataset_finale, field = 'article')

In [ ]:
kw_res.head()